# マッチング部分のデータの準備

In [1]:
import glob

In [2]:
pyfile = glob.glob('/Users/wintor7/Research/train_data_file/**/*.md', recursive=True)

In [3]:
textfile = 'document.txt'

In [35]:
len(pyfile)

7944

In [36]:
pyfile[0]

'/Users/wintor7/Research/train_data_file/monte-language/monte/README.md'

In [37]:
for num in range(1000):
  with open(pyfile[num]) as fh:
    lines = fh.readlines()
    with open(textfile,'a') as tf:
      for i in range(len(lines)):
        tf.write(lines[i])

文書の前処理

In [38]:
with open(textfile) as tf:
  lines = tf.readlines()

In [21]:
len(lines)

762629

In [39]:
lines = [i.strip() for i in lines]

In [40]:
lines = [i.strip('<h1>' '</h1>' '<h2>' '</h2>' '<h3>' '</h3>' '```') for i in lines]

In [41]:
lines = [i for i in lines if i != '']

In [ ]:
with open('token.txt','a') as f:
  for i in range(len(lines)):
    f.write(lines[i]+'\n')

nltkライブラリーのstopwordsを利用する

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wintor7/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/wintor7/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
from nltk.corpus import stopwords
import stopword.stopword as sw
import re

トークンの前処理

In [9]:
with open('token.txt','r') as f:
  lines = f.readlines()

In [10]:
len(lines)

448543

In [11]:
lines[:10]

['Read multiple text files, extract words, and create a dictionary that lists which words are in which file\n',
 'index generator\n',
 'step 1: Read the target files (file1.txt to file3.txt) and prepare "which keyword is included in which file". Let\'s call this an index.\n',
 'For example, the top portion of the diagram above shows the contents of three files. Also, "a 3" at the bottom means "the keyword a is included in page 3 (file3.txt)". Similarly, "cat 1 3" means "the keyword cat is found on pages 1 and 3 (file1.txt and file3.txt)".\n',
 'In this way, let\'s process it so that "which page contains a certain keyword in the file" can be checked.\n',
 'step 2: Return the generated index.\n',
 'matching part\n',
 'step 1: Get the file name containing the keyword based on the index and the specified keyword.\n',
 'For example, file1.txt and file3.txt correspond to "cat" in the schematic diagram. In the case of "dog", file2.txt and file3.txt are applicable.\n',
 'step 2: From the list 

In [12]:
for i in range(len(lines)):
  lines[i] = lines[i].strip()
  lines[i] = lines[i].replace('  ',' ')
  lines[i] = lines[i].replace('.','').replace(',','')
  lines[i] = lines[i].split(' ')

In [13]:
len(lines)

448543

In [14]:
lines = lines[:300000]

In [18]:
stop_words = set(stopwords.words('english'))
lines = [[word for word in line if word not in stop_words]for line in lines]
lines = [[word for word in line if word not in sw.punctuation]for line in lines]
lines = [[word for word in line if not word.isdigit()]for line in lines]
lines = [[word for word in line if not re.search('http', word)]for line in lines]
lines = [[word for word in line if re.search('^([A-Z]|[a-z]|[_]+[a-z]|[A-Z]|[_])+([0-9])*$',word) ]for line in lines]
lines = [line for line in lines if line]

In [19]:
lines[:2000]

[['Read',
  'multiple',
  'text',
  'files',
  'extract',
  'words',
  'create',
  'dictionary',
  'lists',
  'words',
  'file'],
 ['index', 'generator'],
 ['step',
  'Read',
  'target',
  'files',
  'prepare',
  'keyword',
  'included',
  'call',
  'index'],
 ['For',
  'example',
  'top',
  'portion',
  'diagram',
  'shows',
  'contents',
  'three',
  'files',
  'Also',
  'bottom',
  'means',
  'keyword',
  'included',
  'page',
  'Similarly',
  'means',
  'keyword',
  'cat',
  'found',
  'pages'],
 ['In', 'way', 'process', 'page', 'contains', 'certain', 'keyword', 'checked'],
 ['step', 'Return', 'generated', 'index'],
 ['matching', 'part'],
 ['step',
  'Get',
  'file',
  'name',
  'containing',
  'keyword',
  'based',
  'index',
  'specified',
  'keyword'],
 ['For',
  'example',
  'correspond',
  'schematic',
  'diagram',
  'In',
  'case',
  'applicable'],
 ['step',
  'From',
  'list',
  'files',
  'obtained',
  'step',
  'select',
  'pages',
  'contain',
  'specified',
  'keyword'],

In [20]:
len(lines)

220406

マッチング部分のWord2Vecモデルの訓練

In [21]:
import gensim

In [51]:
model = gensim.models.Word2Vec(lines, min_count=5,window=5,epochs=7)

In [52]:
model.save('model/w2v_doc.model')

In [53]:
model = gensim.models.Word2Vec.load('model/w2v_doc.model')

In [54]:
len(model.wv.key_to_index)

12150

In [55]:
test = {'index': 60,
  'filenames': 36,
  'fh': 23,
  'filename': 6,
  'words': 8,
  'file': 7,
  'sentence': 0}

In [56]:
test.keys()


dict_keys(['index', 'filenames', 'fh', 'filename', 'words', 'file', 'sentence'])

In [57]:
x = model.wv.most_similar('index', topn=10)

In [58]:
for i in x:
  print(i)

('element', 0.8071488738059998)
('inserted', 0.6688207387924194)
('slice', 0.6410013437271118)
('ARGs', 0.640264630317688)
('counter', 0.6337505578994751)
('array', 0.613696813583374)
('elements', 0.5949238538742065)
('empty', 0.5941079258918762)
('mask', 0.5936033725738525)
('chain', 0.5935707092285156)


FastTextモデルの試す（廃棄）

In [51]:
ft_model = gensim.models.FastText(lines,window=7,min_count=5,min_n=3,max_n=5)

In [52]:
ft_model.save('model/ft_doc.model')

In [56]:
ft_model.wv.similar_by_word('fh',topn=10)

[('changed', 0.2828810214996338),
 ('modify', 0.28169599175453186),
 ('otherwise', 0.2740412950515747),
 ('moved', 0.26954323053359985),
 ('modifed', 0.26654428243637085),
 ('within', 0.25990766286849976),
 ('unchanged', 0.25960099697113037),
 ('attempt', 0.2587338984012604),
 ('su10', 0.257865846157074),
 ('imported', 0.2566903233528137)]

In [169]:
ft_model.wv.similarity('search_result','search_list')

0.69735295

In [153]:
len(ft_model.wv.index_to_key)

20664

In [157]:
len(ft_model.wv.buckets_word)

20664